In [182]:
import numpy as np
from sklearn.base import BaseEstimator

class LinearClassifier(BaseEstimator):
    """
    General class for binary linear classifiers. Implements the predict
    function, which is the same for all binary linear classifiers. There are
    also two utility functions.
    """

    def decision_function(self, X):
        """
        Computes the decision function for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """
        return X.dot(self.w)

    def predict(self, X):
        """
        Predicts the outputs for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """

        # First compute the output scores
        scores = self.decision_function(X)

        # Select the positive or negative class label, depending on whether
        # the score was positive or negative.
        out = np.select([scores >= 0.0, scores < 0.0],
                        [self.positive_class,
                         self.negative_class])
        return out

    def find_classes(self, Y):
        """
        Finds the set of output classes in the output part Y of the training set.
        If there are exactly two classes, one of them is associated to positive
        classifier scores, the other one to negative scores. If the number of
        classes is not 2, an error is raised.
        """
        classes = sorted(set(Y))
        if len(classes) != 2:
            raise Exception("this does not seem to be a 2-class problem")
        self.positive_class = classes[1]
        self.negative_class = classes[0]

    def encode_outputs(self, Y):
        """
        A helper function that converts all outputs to +1 or -1.
        """
        return np.array([1 if y == self.positive_class else -1 for y in Y])


class PegasusSVC(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter  = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
    
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.lambda_ = 1/n_samples

        # Pegasus algorithm:
        for t in range(1,self.n_iter+1):

            idx = np.random.randint(n_samples)

            eta = 1/(self.lambda_*t)

            x, y = X[idx], Ye[idx]

            # Compute the output score for this instance.
            score = self.w.dot(x)
            # If there was an error, update the weights.
            self.w *= 1-eta*self.lambda_
            
            if y*score < 1:
                self.w += x*eta*y

class PegasusLR(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter  = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
    
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.lambda_ = 1/n_samples

        # Pegasus algorithm:
        for t in range(1,self.n_iter+1):

            idx = np.random.randint(n_samples)

            eta = 1/(self.lambda_*t)

            x, y = X[idx], Ye[idx]
            
            # calculate gradient of log Loss
            dLoss = -y/(1+np.exp(y*(self.w*x)))
            # calc gradient
            gradient = self.lambda_*self.w+dLoss
            # update step
            self.w -= eta*gradient
            #self.w = self.w * (1-eta*self.lambda_) - eta*y*x/(1+np.exp(y*(self.w*x)))

In [183]:
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# from aml_perceptron import Perceptron, SparsePerceptron

# This function reads the corpus, returns a list of documents, and a list
# of their corresponding polarity labels. 
def read_data(corpus_file):
    X = []
    Y = []
    with open(corpus_file, encoding='utf-8') as f:
        for line in f:
            _, y, _, x = line.split(maxsplit=3)
            X.append(x.strip())
            Y.append(y)
    return X, Y



    
# Read all the documents.
X, Y = read_data('DAT341-Applied-Machine-Learning/PA4/data/all_sentiment_shuffled.txt')

# Split into training and test parts.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                random_state=0)
# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),
    
    PegasusSVC(n_iter=10000)
)
# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))
# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 0.96 sec.
Accuracy: 0.8044.


In [184]:
# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),
    
    PegasusLR(n_iter=10000)
)
# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))
# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

/tmp/ipykernel_56165/3667498650.py:153: RuntimeWarning: overflow encountered in exp
  dLoss = -y/(1+np.exp(y*(self.w*x)))


Training time: 1.16 sec.
Accuracy: 0.4964.


In [185]:
import scipy.linalg.blas as blas

class PegasusSVC_BLAS(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter  = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
    
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.lambda_ = 1/n_samples

        # Pegasus algorithm:
        for t in range(1,self.n_iter+1):

            idx = np.random.randint(n_samples)

            eta = 1/(self.lambda_*t)

            x, y = X[idx], Ye[idx]

            # Compute the output score for this instance.
            score = blas.ddot(self.w,x)

            # If there was an error, update the weights.
            
            eta_lam = (1-eta*self.lambda_)

            if y*score < 1:
                blas.dscal(eta_lam, self.w)
                blas.daxpy(x, self.w, a=eta*y)
                
            else:
                blas.dscal(eta_lam, self.w)

def add_sparse_to_dense(x, w, factor):
    """
    Adds a sparse vector x, scaled by some factor, to a dense vector.
    This can be seen as the equivalent of w += factor * x when x is a dense
    vector.
    """
    w[x.indices] += factor * x.data

def sparse_dense_dot(x, w):
    """
    Computes the dot product between a sparse vector x and a dense vector w.
    """
    return np.dot(w[x.indices], x.data)

class SparsePegasusSVC(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm,
    assuming that the input feature matrix X is sparse.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # Initialize the weight vector to all zeros.
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
         
        # set lambda
        self.lambda_ = 1/n_samples

        # pegasus algorothm
        for t in range(1,self.n_iter+1):

            idx = np.random.randint(n_samples)

            eta = 1/(self.lambda_*t)

            x, y = X[idx], Ye[idx]

            # Compute the output score for this instance.
            score = sparse_dense_dot(x,self.w)

            # If there was an error, update the weights.
            
            self.w *= 1-eta*self.lambda_
            if y*score < 1:
                add_sparse_to_dense(x, self.w, eta*y)

In [186]:
# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(ngram_range=(1,2)),
    Normalizer(),
    PegasusSVC(n_iter=10000)
)
# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
train_time = t1-t0
print('Training time: {:.2f} sec.'.format(train_time))

# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 13.82 sec.
Accuracy: 0.8330.


In [187]:
# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(ngram_range=(1,2)),
    Normalizer(),
    PegasusSVC_BLAS(n_iter=10000)
)
# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
train_time_blas = t1-t0
print('Training time: {:.2f} sec.'.format(train_time_blas))

# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 10.06 sec.
Accuracy: 0.8519.


In [188]:
pipeline = make_pipeline(
    TfidfVectorizer(ngram_range=(1,2)),
    Normalizer(),
    SparsePegasusSVC(n_iter=10000)
)
# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
train_time_sparse = t1-t0
print('Training time: {:.2f} sec.'.format(train_time_sparse))
# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 5.41 sec.
Accuracy: 0.8401.


In [189]:
class SparsePegasusSVC2(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm,
    assuming that the input feature matrix X is sparse.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # Initialize the weight vector to all zeros.
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
         
        # set lambda
        self.lambda_ = 1/n_samples
        a = 1
        # pegasus algorothm
        for t in range(1,self.n_iter+1):

            idx = np.random.randint(n_samples)

            eta = 1/(self.lambda_*t)

            x, y = X[idx], Ye[idx]

            # Compute the output score for this instance.
            score = a * sparse_dense_dot(x,self.w)

            # If there was an error, update the weights.
            a *= (1-eta*self.lambda_)
            self.w += (eta*y/a)*x
            
        self.w *= a

In [190]:
pipeline = make_pipeline(
    TfidfVectorizer(ngram_range=(1,2)),
    Normalizer(),
    SparsePegasusSVC2(n_iter=10000)
)
# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
train_time_sparse = t1-t0
print('Training time: {:.2f} sec.'.format(train_time_sparse))
# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

/tmp/ipykernel_56165/682856949.py:47: RuntimeWarning: divide by zero encountered in scalar divide
  self.w += (eta*y/a)*x


IndexError: index 192289 is out of bounds for axis 0 with size 1

In [191]:
from IPython.display import Markdown as md

md("""Train time
   |Pre-optimization|BLAS|Sparse|
   |--|--|--|
   |%.2f|%.2f|%.2f| """ %(train_time,train_time_blas,train_time_sparse))

Train time
   |Pre-optimization|BLAS|Sparse|
   |--|--|--|
   |13.82|10.06|5.41| 